In [ ]:
import torch

In [ ]:
import torch
import matplotlib.pyplot as plt
import os
from get_NN_dataset import CustomDataset

# ────────────────────────────────────────────────────────────────
# 1.  File sanity‑check
# ────────────────────────────────────────────────────────────────
filename = "10_Thousand_samples_LP_DEG_SAC.pt"
if not os.path.isfile(filename) or os.path.getsize(filename) == 0:
    raise FileNotFoundError(f"Dataset file '{filename}' is missing or empty.")

# ────────────────────────────────────────────────────────────────
# 2.  Load (explicitly allow full pickle)
#     • weights_only=False  restores the pre‑2.6 behaviour.
#     • map_location='cpu'  keeps everything on CPU by default.
# ────────────────────────────────────────────────────────────────
filename = "10_Thousand_samples_LP_DEG_SAC.pt"
dataset = torch.load(filename, weights_only=False, map_location="cpu")

# ────────────────────────────────────────────────────────────────
# 3.  Normalise so we always have something index‑able
#     * If the object already acts like a Dataset (has __getitem__)
#       we leave it untouched.
#     * If it’s a dict with plain tensors we wrap it in TensorDataset.
# ────────────────────────────────────────────────────────────────
if isinstance(dataset, dict):
    if {"data", "labels"} <= dataset.keys():
        data_tensor   = dataset["data"]
        labels_tensor = dataset["labels"]
    else:
        raise ValueError(
            f"Expected keys {{'data', 'labels'}}, got {list(dataset.keys())}"
        )
    # Guarantee a channel dim: (N, H, W) → (N, 1, H, W)
    if data_tensor.ndim == 3:
        data_tensor = data_tensor.unsqueeze(1)
    dataset = torch.utils.data.TensorDataset(data_tensor, labels_tensor)

# ────────────────────────────────────────────────────────────────
# 4.  Quick inspection
# ────────────────────────────────────────────────────────────────
print("Samples :", len(dataset))
print("One item :", dataset[0][0].shape, type(dataset[0][1]))

# ────────────────────────────────────────────────────────────────
# 5.  Visualise a single sample
# ────────────────────────────────────────────────────────────────
img, lbl = dataset[0]
plt.figure(figsize=(4, 4))
plt.imshow(img.reshape([8,4]).squeeze().numpy(), cmap="gray", aspect="auto")
plt.title(f"Label: {int(lbl)}")
plt.axis("off")
plt.show()

# ────────────────────────────────────────────────────────────────
# 6.  Visualise the first 10 samples as a heat‑map grid
# ────────────────────────────────────────────────────────────────
rows, cols = 2, 5
fig, axes = plt.subplots(rows, cols, figsize=(12, 5))
axes = axes.flatten()

for i in range(rows * cols):
    img, lbl = dataset[i]
    axes[i].imshow(img.squeeze().numpy(), cmap="hot", aspect="auto")
    axes[i].set_title(f"Label: {int(lbl)}")
    axes[i].axis("off")

plt.tight_layout()
plt.show()


In [ ]:
import torch
import matplotlib.pyplot as plt
import os
filename = "/content/sample_data/10_Thousand_samples_LP_DEG_SAC.pt"

if not os.path.isfile(filename) or os.path.getsize(filename) == 0:
    raise FileNotFoundError(f"Dataset file '{filename}' is missing or empty.")

# 1. Load without the new‑format flag
dataset = torch.load(filename)          # <= remove weights_only

# 2. Inspect what we actually got
print("Loaded object type:", type(dataset))

# If it’s a dict with ‘data’ and ‘labels’ tensors:
if isinstance(dataset, dict) and {'data', 'labels'} <= dataset.keys():
    data_tensor, label_tensor = dataset['data'], dataset['labels']
    print("Data shape:", data_tensor.shape, "Labels shape:", label_tensor.shape)

    # Visualise the first sample
    img = data_tensor[0]
    lbl = int(label_tensor[0])          # makes sure it’s a Python int
else:
    # Assume it’s already a list‑like of (img, label) pairs
    img, lbl = dataset[0]
    if isinstance(lbl, torch.Tensor):
        lbl = int(lbl)

# 3. Plot
plt.figure()
plt.imshow(img.squeeze(), cmap='gray', aspect='auto')  # squeeze in case there’s a 1‑channel dim
plt.title(f"Label: {lbl}")
plt.axis('off')
plt.show()


In [3]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import  transforms # ,datasets
from get_NN_dataset import CustomDataset, generate_dataset

# ---------- Data ----------
class ToBinaryTensor:
    def __call__(self, img):
        t = transforms.functional.to_tensor(img)      # [0,1]
        return torch.where(t > 0.5, 1.0, -1.0)        # {-1, +1}

filename = "new_10_Million_samples_LP_DEG_SAC.pt"
dataset = torch.load(filename, weights_only=False, map_location="cpu")


dataset.data = torch.stack(dataset.data)  # Convert list of tensors to a single tensor
dataset.data = torch.stack(dataset.labels)  # Convert list of tensors to a single tensor

transform = transforms.Compose([ToBinaryTensor()])
# Set your desired train-test ratio (e.g., 0.8 for 80% train, 20% test)
train_test_ratio = 0.8

# Calculate sizes
total_size = len(dataset)  # Assuming dataset.data is a tensor of shape (N, C, H, W)
train_size = int(total_size * train_test_ratio)
test_size = total_size - train_size
print(f"Total samples: {total_size}, Train samples: {train_size}, Test samples: {test_size}")
# Split the dataset
train_ds, test_ds = random_split(dataset, [train_size, test_size])

train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl  = DataLoader(test_ds,  batch_size=256)

# ---------- Model ----------
class BinaryMLP(nn.Module):
    def __init__(self, hidden_dim: int = 256):
        super().__init__()
        self.flatten = nn.Flatten()
        self.inl = nn.Linear(256, hidden_dim) # hidden layer
        self.activation = nn.ReLU()
        self.outl = nn.Linear(hidden_dim, 9)   # output

    def forward(self, x):
        if self.training:
          temp = self.flatten(x)
          temp = self.inl(temp)
          temp = self.activation(temp)
          return self.outl(temp)
        else:
          temp = self.flatten(x)
          temp = torch.where(temp > 0, 1.0, -1.0)
          temp = self.inl(temp)
          temp = self.activation(temp)
          temp = torch.where(temp > 0, 1.0, -1.0)
          return self.outl(temp)

model = BinaryMLP(hidden_dim=256).to("cuda" if torch.cuda.is_available() else "cpu")

# ---------- Training loop ----------
loss_fn   = nn.CrossEntropyLoss()  # works on raw logits
optimizer = optim.Adam(model.parameters(), lr=1e-3)
device    = next(model.parameters()).device

def accuracy(loader):
    correct = total = 0
    for x, y in loader:
        logits = model(x.to(device))
        pred   = logits.argmax(dim=1)
        correct += (pred.cpu() == y).sum().item()
        total   += y.size(0)
    return correct / total

epochs = 5
for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0
    for x, y in train_dl:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        loss = loss_fn(model(x), y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * y.size(0)

    train_acc = accuracy(train_dl)
    model.eval()
    with torch.no_grad():
      test_acc  = accuracy(test_dl)
    print(f"Epoch {epoch}: "
          f"loss={running_loss/len(train_ds):.4f}  "
          f"train_acc={train_acc:.3%}  test_acc={test_acc:.3%}")

Total samples: 184474, Train samples: 147579, Test samples: 36895


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x9 and 256x256)

In [4]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# ---------- Data ----------
class ToBinaryTensor:
    def __call__(self, img):
        t = transforms.functional.to_tensor(img)      # [0,1]
        return torch.where(t > 0.5, 1.0, -1.0)        # {-1, +1}

transform = transforms.Compose([ToBinaryTensor()])
train_ds = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_ds  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_dl = DataLoader(train_ds, batch_size=256, shuffle=True)
test_dl  = DataLoader(test_ds,  batch_size=256)

# ---------- Model ----------
class Binarize(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return torch.where(input > 0, 1.0, -1.0)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.clamp(-1,1)

class BinLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(BinLinear, self).__init__()
        # Define parameters (weights & bias for example)
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bn = nn.BatchNorm1d(num_features=out_features)

    def forward(self, x):
        # Custom operation: simple linear layer here
        preBn = Binarize.apply(x) @ Binarize.apply(self.weight.t())
        postBn = self.bn(preBn)
        return torch.nn.functional.hardtanh(postBn)

class CustomBinaryMLP(nn.Module):
    def __init__(self, hidden_dim: int = 256):
        super().__init__()
        self.flatten = nn.Flatten()              # 28×28 → 784
        self.inl = BinLinear(784, hidden_dim) # hidden layer
        self.activation = nn.ReLU()
        self.outl = nn.Linear(hidden_dim, 10)   # output

    def forward(self, x):
        temp = self.flatten(x)
        temp = self.activation(self.inl(temp))
        return self.outl(temp)

model = CustomBinaryMLP(hidden_dim=256).to("cuda" if torch.cuda.is_available() else "cpu")

# ---------- Training loop ----------
loss_fn   = nn.CrossEntropyLoss()  # works on raw logits
optimizer = optim.Adam(model.parameters(), lr=1e-3)
device    = next(model.parameters()).device

def accuracy(loader):
    correct = total = 0
    for x, y in loader:
        logits = model(x.to(device))
        pred   = logits.argmax(dim=1)
        correct += (pred.cpu() == y).sum().item()
        total   += y.size(0)
    return correct / total

epochs = 5
for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0
    for x, y in train_dl:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        loss = loss_fn(model(x), y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * y.size(0)

    train_acc = accuracy(train_dl)
    model.eval()
    with torch.no_grad():
      test_acc  = accuracy(test_dl)
    print(f"Epoch {epoch}: "
          f"loss={running_loss/len(train_ds):.4f}  "
          f"train_acc={train_acc:.3%}  test_acc={test_acc:.3%}")

100%|██████████| 9.91M/9.91M [00:02<00:00, 3.33MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 187kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 2.09MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 12.2MB/s]


Epoch 1: loss=0.9492  train_acc=86.780%  test_acc=87.540%
Epoch 2: loss=0.4181  train_acc=89.922%  test_acc=90.150%
Epoch 3: loss=0.3321  train_acc=91.327%  test_acc=91.400%
Epoch 4: loss=0.2900  train_acc=92.228%  test_acc=91.900%
Epoch 5: loss=0.2614  train_acc=92.932%  test_acc=92.650%


In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# ---------- Data ----------
class ToBinaryTensor:
    def __call__(self, img):
        t = transforms.functional.to_tensor(img)      # [0,1]
        return torch.where(t > 0.5, 1.0, -1.0)        # {-1, +1}

transform = transforms.Compose([ToBinaryTensor()])

train_ds = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_ds  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl  = DataLoader(test_ds,  batch_size=256)

# ---------- Model ----------
class Binarize(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return torch.where(input > 0, 1.0, -1.0)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.clamp(-1,1)

class BinLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(BinLinear, self).__init__()
        # Define parameters (weights & bias for example)
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bn = nn.BatchNorm1d(num_features=out_features)

    def forward(self, x):
        # Custom operation: simple linear layer here
        preBn = Binarize.apply(x) @ Binarize.apply(self.weight.t())
        postBn = self.bn(preBn)
        return torch.nn.functional.hardtanh(postBn)

class CustomBinaryMLP(nn.Module):
    def __init__(self, hidden_dim: int = 256):
        super().__init__()
        self.flatten = nn.Flatten()              # 28×28 → 784
        self.inl = BinLinear(784, hidden_dim) # hidden layer
        self.activation = nn.ReLU()
        self.outl = nn.Linear(hidden_dim, 10)   # output

    def forward(self, x):
        temp = self.flatten(x)
        temp = self.activation(self.inl(temp))
        return self.outl(temp)

model = CustomBinaryMLP(hidden_dim=256).to("cuda" if torch.cuda.is_available() else "cpu")

# ---------- Training loop ----------
loss_fn   = nn.CrossEntropyLoss()  # works on raw logits
optimizer = optim.Adam(model.parameters(), lr=1e-3)
device    = next(model.parameters()).device

def accuracy(loader):
    correct = total = 0
    for x, y in loader:
        logits = model(x.to(device))
        pred   = logits.argmax(dim=1)
        correct += (pred.cpu() == y).sum().item()
        total   += y.size(0)
    return correct / total

epochs = 5
for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0
    for x, y in train_dl:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        loss = loss_fn(model(x), y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * y.size(0)

    train_acc = accuracy(train_dl)
    model.eval()
    with torch.no_grad():
      test_acc  = accuracy(test_dl)
    print(f"Epoch {epoch}: "
          f"loss={running_loss/len(train_ds):.4f}  "
          f"train_acc={train_acc:.3%}  test_acc={test_acc:.3%}")

* generate dataset
* output bits are desicion points - for example LP is 4 different bits for 4 different threshold.
* start from the simplest network

ideas:
metrics are LP, DP, SAC.
LP:
∑_x (-1)^(f(x)⊕u⋅x
DP: max ddt
SAC:


In [ ]:
filename = "/content/sample_data/10_Million_samples_LP_DEG_SAC.pt"
try:
    with open(filename, 'rb') as f:
        # Read the first 100 bytes
        header = f.read(100)
    print("First 100 bytes of the file:")
    print(header)
except FileNotFoundError:
    print(f"Error: File not found at {filename}")
except Exception as e:
    print(f"An error occurred: {e}")